In [9]:
FILE_PATH = ''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        FILE_PATH = os.path.join(dirname, filename)

In [10]:
import tensorflow as tf

# Check if TensorFlow is using GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  2


In [11]:
# Method 2: Using open and close explicitly
file = open(FILE_PATH, 'r')
content = file.read()
file.close()

In [12]:
#Remove punctuations
import re
_content = re.sub(r'[^\w\s]', '', content)
content_clean = _content.lower()[:200000]

In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [14]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([content_clean])
unique_cnt = len(tokenizer.word_index)
print("Total unique words in the document is ", unique_cnt)

Total unique words in the document is  4606


In [15]:
total_sentences = content_clean.split('\n')
print("Total sentences are ",len(total_sentences))

Total sentences are  951


In [16]:
input_sequences = []
for sentence in total_sentences:
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1,len(tokenized_sentence)):
        input_sequences.append(tokenized_sentence[:i+1])

In [17]:
max_len = max([len(x) for x in input_sequences])
print("Size of the longest sentence is ",max_len)

Size of the longest sentence is  351


In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')

In [19]:
padded_input_sequences

array([[  0,   0,   0, ...,   0,   1, 114],
       [  0,   0,   0, ...,   1, 114,  93],
       [  0,   0,   0, ...,   0, 223, 400],
       ...,
       [  0,   0,   0, ..., 326,   5, 244],
       [  0,   0,   0, ...,   5, 244,  96],
       [  0,   0,   0, ..., 244,  96,   2]], dtype=int32)

In [20]:
X = padded_input_sequences[:,:-1]

In [21]:
X.shape

(37641, 350)

In [22]:
y = padded_input_sequences[:,-1]

In [23]:
y.shape

(37641,)

In [24]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=unique_cnt+1)

In [25]:
y.shape

(37641, 4607)

In [26]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

In [27]:
X.shape

(37641, 350)

In [28]:
y.shape

(37641, 4607)

In [ ]:
model = Sequential()
model.add(Embedding(unique_cnt+1, 128, input_shape=(max_len,)))
model.add(LSTM(150))
model.add(Dense(unique_cnt+1, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X,y,epochs=100)

In [29]:
model_two = Sequential()
model_two.add(Embedding(unique_cnt+1, 128, input_shape=(max_len,)))
model_two.add(LSTM(256))
model_two.add(Dense(unique_cnt+1, activation='softmax'))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [30]:
model_two.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [31]:
model_two.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 351, 128)       │       589,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 256)            │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4607)           │     1,183,999 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,167,935 (8.27 MB)

 Trainable params: 2,167,935 (8.27 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_two.fit(X,y,epochs=42)

Epoch 1/42
1177/1177 ━━━━━━━━━━━━━━━━━━━━ 28s 22ms/step - accuracy: 0.0730 - loss: 6.5737
Epoch 2/42
1177/1177 ━━━━━━━━━━━━━━━━━━━━ 26s 22ms/step - accuracy: 0.1138 - loss: 5.6792
Epoch 3/42
1177/1177 ━━━━━━━━━━━━━━━━━━━━ 27s 23ms/step - accuracy: 0.1432 - loss: 5.1374
Epoch 4/42
1177/1177 ━━━━━━━━━━━━━━━━━━━━ 27s 23ms/step - accuracy: 0.1659 - loss: 4.6751
Epoch 5/42
1177/1177 ━━━━━━━━━━━━━━━━━━━━ 27s 23ms/step - accuracy: 0.1936 - loss: 4.2419
Epoch 6/42
1177/1177 ━━━━━━━━━━━━━━━━━━━━ 27s 23ms/step - accuracy: 0.2354 - loss: 3.7855
Epoch 7/42
 648/1177 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - accuracy: 0.3040 - loss: 3.2846

In [ ]:
import time
import numpy as np
text = "what is the fee"

for i in range(10):
  # tokenize
    token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
    padded_token_text = pad_sequences([token_text], maxlen=max_len, padding='pre')
  # predict
    pos = np.argmax(model.predict(padded_token_text))
    for word,index in tokenizer.word_index.items():
        if index == pos:
            text = text + " " + word
            print(text)
            time.sleep(2)